In [1]:
# from torch.utils.data import DataLoader
# from GeoSeg.geoseg.losses import *
# from GeoSeg.geoseg.datasets.uavid_dataset import *
# from GeoSeg.geoseg.models.UNetFormer import UNetFormer
# from GeoSeg.tools.utils import Lookahead
# from GeoSeg.tools.utils import process_model_params

# # training hparam
# max_epoch = 40
# ignore_index = 255
# train_batch_size = 1
# val_batch_size = 8
# lr = 6e-4
# weight_decay = 0.01
# backbone_lr = 6e-5
# backbone_weight_decay = 0.01
# num_classes = len(CLASSES)
# classes = CLASSES

# weights_name = "unetformer-r18-1024-768crop-e40"
# weights_path = "model_weights/uavid/{}".format(weights_name)
# test_weights_name = "last"
# log_name = 'uavid/{}'.format(weights_name)
# monitor = 'val_mIoU'
# monitor_mode = 'max'
# save_top_k = 1
# save_last = True
# check_val_every_n_epoch = 1
# pretrained_ckpt_path = None # the path for the pretrained model weight
# gpus = 'auto'  # default or gpu ids:[0] or gpu nums: 2, more setting can refer to pytorch_lightning
# resume_ckpt_path = None  # whether continue training with the checkpoint, default None

# #  define the network
# net = UNetFormer(num_classes=num_classes)
# # define the loss
# loss = UnetFormerLoss(ignore_index=ignore_index)

# use_aux_loss = True

# # define the dataloader

# train_dataset = UAVIDDatasetV2(data_root='C:/Users/Franklin/code/Aerial image segmentation/Code/data/uavid/uavid_train_val', img_dir='images', mask_dir='masks',
#                              mode='train', mosaic_ratio=0.25, transform=train_aug, img_size=(1024, 1024))

# val_dataset = UAVIDDataset(data_root='C:/Users/Franklin/code/Aerial image segmentation/Code/data/uavid/val', img_dir='images', mask_dir='masks', mode='val',
#                            mosaic_ratio=0.0, transform=val_aug, img_size=(1024, 1024))


# train_loader = DataLoader(dataset=train_dataset,
#                           batch_size=train_batch_size,
#                           num_workers=4,
#                           pin_memory=True,
#                           shuffle=True,
#                           drop_last=True)

# val_loader = DataLoader(dataset=val_dataset,
#                         batch_size=val_batch_size,
#                         num_workers=4,
#                         shuffle=False,
#                         pin_memory=True,
#                         drop_last=False)

In [2]:
# Inference code
from train.static_train import *
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def uavid2rgb(mask):
    # dictionary mapping
    h, w = mask.shape[0], mask.shape[1]
    # print(h, w)
    mask_rgb = np.zeros(shape=(h, w, 3), dtype=np.uint8)
    mask_convert = mask[np.newaxis, :, :]
    # print(mask_convert.shape)
    # print(mask_rgb.shape)
    mask_rgb[np.all(mask_convert == 0, axis=0)] = [128, 0, 0]
    mask_rgb[np.all(mask_convert == 1, axis=0)] = [128, 64, 128]
    mask_rgb[np.all(mask_convert == 2, axis=0)] = [0, 128, 0]
    mask_rgb[np.all(mask_convert == 3, axis=0)] = [128, 128, 0]
    mask_rgb[np.all(mask_convert == 4, axis=0)] = [64, 0, 128]
    mask_rgb[np.all(mask_convert == 5, axis=0)] = [192, 0, 192]
    mask_rgb[np.all(mask_convert == 6, axis=0)] = [64, 64, 0]
    mask_rgb[np.all(mask_convert == 7, axis=0)] = [0, 0, 0]
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    return mask_rgb

class Args:
    image_path = "C:/Users/Franklin/code/Aerial image segmentation/Code/data/uavid/uavid_test"
    config_path = "C:/Users/Franklin/code/Aerial image segmentation/Code/GeoSeg/config/uavid/unetformer.py"
    output_path = "C:/Users/Franklin/code/Aerial image segmentation/Code/fig_results/uavid/unetformer_r18"
    tta = "lr"
    patch_height = 1152
    patch_width = 1024
    batch_size = 8
    dataset = "uavid"

def load_checkpoint(checkpoint_path, model):
    pretrained_dict = torch.load(checkpoint_path)['model_state_dict']
    model_dict = model.state_dict()
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)

    return model


In [3]:
# img_path = "C:/Users/Franklin/code/Aerial image segmentation/Code/data/uavid/uavid_test/seq22/Images/000000.png"
# a = cv2.imread(img_path, cv2.IMREAD_COLOR)
# a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
# dataset, width_pad, height_pad, output_width, output_height, img_pad, img_shape = process_frame(a, (1152,1024))

In [25]:
# Processing code
import cv2
import numpy as np
import albumentations as albu
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import tqdm

class InferenceDataset(Dataset):
    def __init__(self, tile_list=None, transform=albu.Normalize()):
        self.tile_list = tile_list
        self.transform = transform

    def __getitem__(self, index):
        img = self.tile_list[index]
        img_id = index
        aug = self.transform(image=img)
        img = aug['image']
        img = torch.from_numpy(img).permute(2, 0, 1).float()
        results = dict(img_id=img_id, img=img)
        return results

    def __len__(self):
        return len(self.tile_list)
    

def get_img_padded(image, patch_size):
    oh, ow = image.shape[0], image.shape[1]
    rh, rw = oh % patch_size[0], ow % patch_size[1]

    width_pad = 0 if rw == 0 else patch_size[1] - rw
    height_pad = 0 if rh == 0 else patch_size[0] - rh
    # print(oh, ow, rh, rw, height_pad, width_pad)
    h, w = oh + height_pad, ow + width_pad

    pad = albu.PadIfNeeded(min_height=h, min_width=w, border_mode=0,
                            position='bottom_right', value=[0, 0, 0])(image=image)
    img_pad = pad['image']
    return img_pad, height_pad, width_pad

def process_frame(frame, patch_size):
    tile_ls = []
    image_pad, height_pad, width_pad = get_img_padded(frame.copy(), patch_size)
    output_height, output_width = image_pad.shape[0], image_pad.shape[1]

    for x in range(0, output_height, patch_size[0]):
        for y in range(0, output_width, patch_size[1]):
            frame_tile = image_pad[x:x+patch_size[0], y:y+patch_size[1]]
            tile_ls.append(frame_tile)
    dataset = InferenceDataset(tile_list=tile_ls)
    return dataset, width_pad, height_pad, output_width, output_height, image_pad, frame.shape

def apply_mask(frame, seg, alpha = 0.5):
    combined_frame = cv2.addWeighted(frame,alpha, seg, 1 - alpha,0)
    return combined_frame
def processing_4kvideo(video_path, output_path, frame_interval = 5, resized = True, target_size = None):
    args = Args
    config = py2cfg(args.config_path)
    weights_name = "unetformer-r18-1024-768crop-e40"
    weights_path = "model_weights/uavid/{}".format(weights_name)
    test_weights_name = "last"
    path = "c:/Users/Franklin/code/Aerial image segmentation/Code/model_weights/uavid/unetformer-r18-1024-768crop-e40/last.ckpt"
    model = Supervision_Train.load_from_checkpoint(path, config = config)
    model.cuda("cuda")
    model.eval()
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # define video 
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps//frame_interval, target_size)
    patch_size = (args.patch_height, args.patch_width)
    frame_count = 0
    if not resized:
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                break
            if frame_count % frame_interval == 0:
                print("Frame count: ", frame_count)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # frame = torch.tensor(frame).permute(2,1,0)
                dataset, width_pad, height_pad, output_width, output_height, img_pad, img_shape = process_frame(frame, (args.patch_height, args.patch_width))
                
                output_mask = np.zeros(shape=(output_height, output_width), dtype=np.uint8)
                output_tiles = []
                k = 0
                with torch.no_grad():
                    dataloader = DataLoader(dataset=dataset, batch_size=args.batch_size,
                                            drop_last = False, shuffle = False)
                    for inp in tqdm.tqdm(dataloader):
                        raw_pred = model(inp["img"].cuda("cuda"))
                        raw_pred = nn.Softmax(dim=1)(raw_pred)
                        pred = raw_pred.argmax(dim=1)
                        img_ids = inp["img_id"]
                        for i in range(pred.shape[0]):
                            raw_mask = pred[i].cpu().numpy()
                            mask = raw_mask
                            output_tiles.append((mask, img_ids[i].cpu().numpy()))
                # Combine
                for m in range(0, output_height, patch_size[0]):
                    for n in range(0, output_width, patch_size[1]):
                        output_mask[m:m + patch_size[0], n:n + patch_size[1]] = output_tiles[k][0]
                        k = k + 1

                output_mask = output_mask[-img_shape[0]:, -img_shape[1]:]
                # print(output_mask.shape)
                output_mask = uavid2rgb(output_mask)
                # print(output_mask.shape)
                # break
                # write to new video
                # Add alpha as 
                output_mask = apply_mask(frame,output_mask, alpha=0.3)
                out.write(output_mask)
                frame_count += 1
            else:
                frame_count += 1
                continue
    else:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            if frame_count % frame_interval == 0:
                print("Frame count: ", frame_count)
                frames = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                # frame = cv2.resize(frame, target_size)
                # frame = torch.tensor(frame).permute(2,1,0)
                # output_mask = np.zeros(shape=target_size, dtype=np.uint8)
                output_tiles = []
                transform=albu.Normalize()
                frame = transform(image = frames)
                frame = frame["image"]
                frame = torch.from_numpy(frame).permute(2, 0, 1).float().unsqueeze(0)
                # print(frame.shape)
                with torch.no_grad():
                    raw_pred = model(frame.cuda("cuda"))
                    raw_pred = nn.Softmax(dim=1)(raw_pred)
                    pred = raw_pred.argmax(dim=1)[0]
                    pred = pred.cpu().numpy()
                        # for i in range(pred.shape[0]):
                        #     raw_mask = pred[i].cpu().numpy()
                        #     mask = raw_mask
                        #     output_tiles.append((mask, img_ids[i].cpu().numpy()))
                # print(pred.shape)
                output_mask = uavid2rgb(pred)
                # print(frame.cpu().numpy().squeeze(0))
                output_mask = apply_mask(frames,output_mask, alpha=0.3)
                # output_mask = cv2.resize(output_mask, (width, height))
                # print(output_mask)
                # break
                # write to new video
                print(output_mask.shape)
                # print(output_mask)
                out.write(output_mask)
                frame_count += 1
                # break
            else:
                frame_count += 1
                continue
    cap.release()
    out.release()
    # cv2.destroyAllWindows()


In [29]:
video_path = "C:/Users/Franklin/Desktop/Data/Aerial image seg/video/source/alt100deg60(1)-20240514-djiair3.MP4"
output_video_path = "C:/Users/Franklin/Desktop/Data/Aerial image seg/video/mask/alt100deg60(1)-20240514-mask.MP4"
processing_4kvideo(video_path,output_video_path, resized=True, target_size=(3840,2160))

Frame count:  0
(2160, 3840, 3)
Frame count:  5
(2160, 3840, 3)
Frame count:  10
(2160, 3840, 3)
Frame count:  15
(2160, 3840, 3)
Frame count:  20
(2160, 3840, 3)
Frame count:  25
(2160, 3840, 3)
Frame count:  30
(2160, 3840, 3)
Frame count:  35
(2160, 3840, 3)
Frame count:  40
(2160, 3840, 3)
Frame count:  45
(2160, 3840, 3)
Frame count:  50
(2160, 3840, 3)
Frame count:  55
(2160, 3840, 3)
Frame count:  60
(2160, 3840, 3)
Frame count:  65
(2160, 3840, 3)
Frame count:  70
(2160, 3840, 3)
Frame count:  75
(2160, 3840, 3)
Frame count:  80
(2160, 3840, 3)
Frame count:  85
(2160, 3840, 3)
Frame count:  90
(2160, 3840, 3)
Frame count:  95
(2160, 3840, 3)
Frame count:  100
(2160, 3840, 3)
Frame count:  105
(2160, 3840, 3)
Frame count:  110
(2160, 3840, 3)
Frame count:  115
(2160, 3840, 3)
Frame count:  120
(2160, 3840, 3)
Frame count:  125
(2160, 3840, 3)
Frame count:  130
(2160, 3840, 3)
Frame count:  135
(2160, 3840, 3)
Frame count:  140
(2160, 3840, 3)
Frame count:  145
(2160, 3840, 3)
Fr

In [11]:
def process_single(input_path, output_path, target_size):
    cap = cv2.VideoCapture(input_path)
    args = Args
    config = py2cfg(args.config_path)
    weights_name = "unetformer-r18-1024-768crop-e40"
    weights_path = "model_weights/uavid/{}".format(weights_name)
    test_weights_name = "last"
    path = "c:/Users/Franklin/code/Aerial image segmentation/Code/model_weights/uavid/unetformer-r18-1024-768crop-e40/last.ckpt"
    model = Supervision_Train.load_from_checkpoint(path, config = config)
    model.cuda("cuda")
    model.eval()
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read video")
        return
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, target_size)
    # frame = torch.tensor(frame).permute(2,1,0)
    # output_mask = np.zeros(shape=target_size, dtype=np.uint8)
    output_tiles = []
    transform=albu.Normalize()
    frame = transform(image = frame)
    frame = frame["image"]
    frame = torch.from_numpy(frame).permute(2, 0, 1).float().unsqueeze(0)
    # print(frame.shape)
    with torch.no_grad():
        raw_pred = model(frame.cuda("cuda"))
        raw_pred = nn.Softmax(dim=1)(raw_pred)
        pred = raw_pred.argmax(dim=1)[0]
        pred = pred.cpu().numpy()
            # for i in range(pred.shape[0]):
            #     raw_mask = pred[i].cpu().numpy()
            #     mask = raw_mask
            #     output_tiles.append((mask, img_ids[i].cpu().numpy()))
    # print(pred.shape)
    output_mask = uavid2rgb(pred)
    cv2.imwrite(f"{output_path}/segmented_frame.jpg", output_mask)
    # output_mask = cv2.resize(output_mask, (width, height))
    # print(output_mask)
    # break
    # write to new video
    print(output_mask.shape)

In [14]:
video_path = "C:/Users/Franklin/Desktop/Data/Aerial image seg/video/source/alt80deg45(1)-20240514-djiair3.MP4"
output_video_path = "C:/Users/Franklin/Desktop/Data/Aerial image seg/video/mask"
process_single(video_path,output_video_path, target_size=(3840,2160))

(2160, 3840, 3)
